# Data Cleaning and Preprocessing

This notebook's goal is to clean and preprocess the NHANES (August 2021 - August 2023) dataset to prepare it for use in a machine learning model aimed at predicting diabetes risk.

### Key Columns
SEQN - Respondent sequence number

## Table of Contents
1. [Importing Libraries](#import-libraries)
2. [Loading the Dataset](#loading-the-dataset)
3. [Initial Data Exploration](#data-exploration)
4. [Data Cleaning](#data-cleaning)
5. [Data Preprocessing]()
6. [Exploratory Data Analysis (EDA)]#eda)


# Import Libraries 

In [1]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
import sys
import os

# Import functions
notebook_dir = os.getcwd()
src_path = os.path.join(notebook_dir, '../src')
sys.path.append(src_path)
from data_cleaning import filter_columns


# Loading the dataset

In [2]:
folder_path = '../data/raw/csv_files/'

# list csv files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

merged_df = None
# Load each csv file into a pandas dataframe and merge on 'SEQN' column
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)

    if merged_df is None:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on='SEQN', how='inner')


# Data Exploration

In [3]:
# Check merged dataframe structure
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6337 entries, 0 to 6336
Data columns (total 80 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      6337 non-null   float64
 1   ALQ111    5481 non-null   float64
 2   ALQ121    4922 non-null   float64
 3   ALQ130    4069 non-null   float64
 4   ALQ142    4082 non-null   float64
 5   ALQ270    2366 non-null   float64
 6   ALQ280    2362 non-null   float64
 7   ALQ151    4901 non-null   float64
 8   ALQ170    2358 non-null   float64
 9   BPQ020    6337 non-null   float64
 10  BPQ030    2322 non-null   float64
 11  BPQ150    2323 non-null   float64
 12  BPQ080    6337 non-null   float64
 13  BPQ101D   6337 non-null   float64
 14  DBQ010    0 non-null      float64
 15  DBD030    0 non-null      float64
 16  DBD041    0 non-null      float64
 17  DBD050    0 non-null      float64
 18  DBD055    0 non-null      float64
 19  DBD061    0 non-null      float64
 20  DBQ073A   0 non-null      floa

In [4]:
# Show the first 5 rows of the merged dataframe
merged_df.head()

,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ151,ALQ170,BPQ020,...,SMQ040,SMD641,SMD650,SMD100MN,SMQ621,SMD630,SMAQUEX2,WTPH2YR,LBXTC,LBDTCSI
0,130378.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,3.0,NaN,NaN,NaN,NaN,NaN,1.0,56042.129410,264.0,6.83
1,130379.0,1.0,2.0,3.0,0.0,NaN,NaN,2.0,NaN,1.0,...,3.0,NaN,NaN,NaN,NaN,NaN,1.0,37435.705647,214.0,5.53
2,130380.0,1.0,10.0,1.0,0.0,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,85328.844519,187.0,4.84
3,130386.0,1.0,4.0,2.0,10.0,0.0,10.0,2.0,0.0,2.0,...,3.0,NaN,NaN,NaN,NaN,NaN,1.0,44526.214135,183.0,4.73
4,130387.0,1.0,0.0,NaN,NaN,NaN,NaN,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,22746.296353,203.0,5.25


In [5]:
# Check for missing values
merged_df.isnull().sum()

SEQN           0
ALQ111       856
ALQ121      1415
ALQ130      2268
ALQ142      2255
            ... 
SMD630      6337
SMAQUEX2       0
WTPH2YR        0
LBXTC        620
LBDTCSI      620
Length: 80, dtype: int64

In [6]:
# Summary statistics
merged_df.describe()   

,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ151,ALQ170,BPQ020,...,SMQ040,SMD641,SMD650,SMD100MN,SMQ621,SMD630,SMAQUEX2,WTPH2YR,LBXTC,LBDTCSI
count,6337.000000,5481.000000,4922.000000,4069.000000,4082.000000,2366.000000,2362.000000,4901.000000,2358.000000,6337.000000,...,2518.000000,191.000000,912.000000,912.000000,0.0,0.0,6337.0,6337.000000,5717.000000,5717.000000
mean,136348.738362,1.109104,5.030679,5.842959,4.742283,4.838123,3.545301,1.821261,4.396098,1.637841,...,2.339952,14.015707,18.188596,0.418860,NaN,NaN,1.0,40024.873091,186.974637,4.835153
std,3439.132476,0.385114,4.314321,54.996448,7.326042,7.785415,7.133496,0.458352,45.252453,0.516130,...,0.901815,15.099835,80.499562,0.633839,NaN,NaN,0.0,31504.418240,42.666181,1.103440
min,130378.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,0.000000,1.000000,0.000000,NaN,NaN,1.0,0.000000,62.000000,1.600000
25%,133337.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,...,1.000000,4.000000,5.000000,0.000000,NaN,NaN,1.0,19228.917397,157.000000,4.060000
50%,136397.000000,1.000000,5.000000,2.000000,4.000000,4.000000,0.000000,2.000000,1.000000,2.000000,...,3.000000,12.000000,10.000000,0.000000,NaN,NaN,1.0,31687.062804,184.000000,4.760000
75%,139310.000000,1.000000,8.000000,3.000000,9.000000,9.000000,7.000000,2.000000,2.000000,2.000000,...,3.000000,20.000000,20.000000,1.000000,NaN,NaN,1.0,50785.688486,213.000000,5.510000
max,142310.000000,9.000000,99.000000,999.000000,99.000000,99.000000,99.000000,9.000000,999.000000,9.000000,...,3.000000,99.000000,999.000000,9.000000,NaN,NaN,1.0,241728.857241,438.000000,11.330000


# Data Cleaning

## Get rid of Irrelevant Columns

Columns to be used: 

In [7]:
filtered_df = filter_columns(merged_df)
filtered_df.describe()

,SEQN,LBDTCSI,DIQ010,DID040,DIQ160,DBQ930,DBQ940,DBQ945,BPQ020,BPQ080,...,DPQ030,DPQ040,DPQ050,DPQ060,DPQ070,DPQ080,DPQ090,DPQ100,SMQ040,SMD641
count,6337.000000,5717.000000,6337.000000,870.000000,5242.000000,6337.000000,6337.000000,6337.000000,6337.000000,6337.000000,...,5516.000000,5514.000000,5513.000000,5510.000000,5508.000000,5508.000000,5506.000000,4167.000000,2518.000000,191.000000
mean,136348.738362,4.835153,1.899006,65.454023,1.875811,1.326180,1.326022,1.480511,1.637841,1.640997,...,0.793510,0.860174,0.504263,0.411071,0.394880,0.216594,0.082092,0.479242,2.339952,14.015707
std,3439.132476,1.103440,0.412357,124.466982,0.452716,0.468851,0.478126,0.508427,0.516130,0.734506,...,1.003693,0.965127,0.860577,0.786979,0.767524,0.685077,0.438164,0.740106,0.901815,15.099835
min,130378.000000,1.600000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,133337.000000,4.060000,2.000000,40.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
50%,136397.000000,4.760000,2.000000,50.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,12.000000
75%,139310.000000,5.510000,2.000000,60.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,3.000000,20.000000
max,142310.000000,11.330000,9.000000,999.000000,9.000000,2.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,3.000000,99.000000


## Handle Missing Values

### Columns that are more than 50% null

In [8]:
total_rows = len(filtered_df)

# Count null values and count of values equal to 999 and 99 for each column
null_counts = filtered_df.isnull().sum()
more_null = (filtered_df == 999).sum()
nine_nine_null = (filtered_df == 99).sum()

cols_to_drop = []

for col in filtered_df.columns:
    if null_counts[col] > 0.5 * total_rows or more_null[col] > 0.5 * total_rows or nine_nine_null[col] > 0.5 * total_rows:
        cols_to_drop.append(col)

cols_to_drop

['DID040', 'BPQ150', 'PAD810U', 'ALQ280', 'SMQ040', 'SMD641']

#### Columns with over half null values

1. DID040: Age when first told you had diabetes
    - Will be left as null as this likely means they do not have diabetes at all.
2. BPQ150: taking high blood pressure medication
    - Missing values will be imputed with mode value.
3. PAD810U: Vigorous Leisure Time Physical Activity (LTPA) unit day/week/month/year
    - Missing values could indivate respondents do not engage in this specific physical activity.
4. ALQ280: # of times 8+ drinks in 1 day/past 12 months
    - Missing data with 0.
5.SMQ040: Do you now smoke cigarettes?
    - Missing values are filled with 0 as it likely means they are non-smokers.
6. SMD641: # days smoked cigarettes during past 30 days
    - Missing values are filled with 0 as it likely means they are non-smokers.


In [12]:
# Replace 99 and 999 with NaN
filtered_df.loc[:, :] = filtered_df.replace([99, 999], np.nan)

fill_vals = {
    'BPQ150': filtered_df['BPQ150'].mode()[0],
    'PAD810U': 0,
    'ALQ280': 0,
    'SMQ040': 0,
    'SMD641': 0 
}

filtered_df.loc[:, :] = filtered_df.fillna(value=fill_vals)

#### Percentage of Null Values in each Column

In [16]:
null_percentage = filtered_df.isnull().mean() * 100
null_percentage[null_percentage > 0]

LBDTCSI      9.783809
DID040      86.507811
DIQ160      17.279470
PAD680       0.094682
PAD790U     20.609121
ALQ130      35.947609
ALQ142      35.774026
INDFMMPC     8.205776
DPQ010      12.908316
DPQ020      12.924097
DPQ030      12.955657
DPQ040      12.987218
DPQ050      13.002998
DPQ060      13.050339
DPQ070      13.081900
DPQ080      13.081900
DPQ090      13.113461
DPQ100      34.243333
dtype: float64